In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


# Approach 1

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from scipy.stats import skew, kurtosis
from sklearn.utils import resample

# Load data
train = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv')
test = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv')
ndvi_cols = [col for col in train.columns if col.endswith('_N')]

# Map class to integer
class_map = {'water': 0, 'impervious': 1, 'farm': 2, 'forest': 3, 'grass': 4, 'orchard': 5}
inv_class_map = {v: k for k, v in class_map.items()}
train['class_code'] = train['class'].map(class_map)

# Upsample minority classes
def balance_classes(df):
    major = df[df['class'] == 'forest']
    dfs = [major]
    for cls in ['farm', 'impervious', 'grass', 'water', 'orchard']:
        minor = df[df['class'] == cls]
        upsampled = resample(minor, replace=True, n_samples=len(major)//2, random_state=42)
        dfs.append(upsampled)
    return pd.concat(dfs)

train = balance_classes(train)

# Impute and feature engineer
def feature_engineer(df):
    X = df[ndvi_cols]
    imputer = KNNImputer(n_neighbors=3)
    X_imp = imputer.fit_transform(X)

    feats = pd.DataFrame()
    feats['mean'] = X_imp.mean(axis=1)
    feats['std'] = X_imp.std(axis=1)
    feats['min'] = X_imp.min(axis=1)
    feats['max'] = X_imp.max(axis=1)
    feats['range'] = feats['max'] - feats['min']
    feats['skew'] = skew(X_imp, axis=1)
    feats['kurtosis'] = kurtosis(X_imp, axis=1)
    feats['trend'] = (X_imp[:, -1] - X_imp[:, 0]) / (X_imp.shape[1])

    # Seasonal features
    feats['early_season'] = X_imp[:, :9].mean(axis=1)
    feats['mid_season'] = X_imp[:, 9:18].mean(axis=1)
    feats['late_season'] = X_imp[:, 18:].mean(axis=1)
    
    return feats

X_train = feature_engineer(train)
X_test = feature_engineer(test)
y_train = train['class_code']

# Standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model: Tuned XGBoost
xgb = XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.03,
    subsample=0.85,
    colsample_bytree=0.9,
    objective='multi:softprob',
    num_class=6,
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train_scaled, y_train)

# Predict
preds = xgb.predict(X_test_scaled)
preds_labels = [inv_class_map[i] for i in preds]

# Submission
submission = pd.DataFrame({
    'ID': test['ID'],
    'class': preds_labels
})
submission.to_csv('submission.csv', index=False)
print("✅ Final submission file created successfully!")


✅ Final submission file created successfully!


# Approach 2

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from scipy.stats import skew, kurtosis

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Load Data
train = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv')
test = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv')
ndvi_cols = [col for col in train.columns if col.endswith('_N')]

# Label Encoding
class_map = {'water': 0, 'impervious': 1, 'farm': 2, 'forest': 3, 'grass': 4, 'orchard': 5}
inv_class_map = {v: k for k, v in class_map.items()}
train['class_code'] = train['class'].map(class_map)

# Balanced Sampling
def balance(df):
    dfs = [df[df['class'] == 'forest']]
    for cls in ['farm', 'impervious', 'grass', 'water', 'orchard']:
        d = df[df['class'] == cls]
        dfs.append(resample(d, replace=True, n_samples=5000, random_state=42))
    return pd.concat(dfs)

train = balance(train)

# Feature Engineering
def features(df):
    X = df[ndvi_cols].copy()
    imputer = KNNImputer(n_neighbors=3)
    X_imp = imputer.fit_transform(X)

    df_feat = pd.DataFrame()
    df_feat['mean'] = X_imp.mean(axis=1)
    df_feat['std'] = X_imp.std(axis=1)
    df_feat['min'] = X_imp.min(axis=1)
    df_feat['max'] = X_imp.max(axis=1)
    df_feat['range'] = df_feat['max'] - df_feat['min']
    df_feat['skew'] = skew(X_imp, axis=1)
    df_feat['kurt'] = kurtosis(X_imp, axis=1)
    df_feat['trend'] = (X_imp[:, -1] - X_imp[:, 0]) / X_imp.shape[1]

    df_feat['early'] = X_imp[:, :9].mean(axis=1)
    df_feat['mid'] = X_imp[:, 9:18].mean(axis=1)
    df_feat['late'] = X_imp[:, 18:].mean(axis=1)

    # Rolling window stats
    for w in [3, 5]:
        roll_mean = pd.DataFrame(X_imp).rolling(window=w, axis=1).mean().mean(axis=1)
        roll_std = pd.DataFrame(X_imp).rolling(window=w, axis=1).std().mean(axis=1)
        df_feat[f'roll{w}_mean'] = roll_mean
        df_feat[f'roll{w}_std'] = roll_std

    return df_feat

# Prepare data
X_train = features(train)
y_train = train['class_code']
X_test = features(test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Stack
xgb = XGBClassifier(n_estimators=500, learning_rate=0.03, max_depth=6, subsample=0.9, colsample_bytree=0.9, objective='multi:softmax', use_label_encoder=False, eval_metric='mlogloss')
lgb = LGBMClassifier(n_estimators=500, learning_rate=0.03, max_depth=6)
cat = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.03, verbose=0)

ensemble = VotingClassifier(
    estimators=[('xgb', xgb), ('lgb', lgb), ('cat', cat)],
    voting='soft'
)

# Train
ensemble.fit(X_train_scaled, y_train)

# Predict
y_pred = ensemble.predict(X_test_scaled)
y_label = [inv_class_map[i] for i in y_pred]

# Submission
sub = pd.DataFrame({'ID': test['ID'], 'class': y_label})
sub.to_csv('submission.csv', index=False)
print("✅ Ensemble submission created.")


/tmp/ipykernel_13/107974898.py:57: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  roll_mean = pd.DataFrame(X_imp).rolling(window=w, axis=1).mean().mean(axis=1)
/tmp/ipykernel_13/107974898.py:58: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  roll_std = pd.DataFrame(X_imp).rolling(window=w, axis=1).std().mean(axis=1)
/tmp/ipykernel_13/107974898.py:57: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  roll_mean = pd.DataFrame(X_imp).rolling(window=w, axis=1).mean().mean(axis=1)
/tmp/ipykernel_13/107974898.py:58: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  roll_std = pd.DataFrame(X_imp).rolling(window=w, axis=1)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005570 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 31159, number of used features: 15
[LightGBM] [Info] Start training from score -1.829665
[LightGBM] [Info] Start training from score -1.829665
[LightGBM] [Info] Start training from score -1.829665
[LightGBM] [Info] Start training from score -1.621189
[LightGBM] [Info] Start training from score -1.829665
[LightGBM] [Info] Start training from score -1.829665
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [